In [1]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
LEGEND_LABELS = [
    "(A) Below boundary (safe)",
    "(B) In zone of uncertainty (increasing risk)",
    "(C) Beyond zone of uncertainty (high risk)",
]

def add_pb_subplot(data, column, fig, subplot='polar', subplot2='polar2', domain_x=[0,1], domain_y=[0,1], max_val=3.6, bg_limit = 3, show_safety_grid=True):
    """
    Adds a polar bar plot to the figure, with the values from the column `column` of the DataFrame `data`.
    The safety grid lines are added as a separate subplot overlaying the polar bar plot (otherwise they would not be visible).
    
    If a second PB subplot is needed, use subplot='polar3' and subplot2='polar4', and so on. The parameters domain_x and domain_y
    determine where the subplot is placed on the whole figure.

    Other parameters:
    - max_val: the maximum value shown in the figure (how far does the r-axis extend)
    - bg_limit: the value of the inner circle (background) of the plot
    - show_safety_grid: whether to show the circular dotted grid lines for the safety zones
    """
    
    # Divide the circle in n parts, where n is the number of categories (number of rows in data)
    n = len(data)
    beginthetas = np.arange(n) * 360 / n
    midthetas = beginthetas + 360/(2*n)
    category_names = data.index.values
    
    background_r = bg_limit

    ## Background
    fig.add_trace(go.Barpolar(
        r=[background_r], theta=[0], width=[360], marker_color='#E9F0FA',
        showlegend=False, subplot=subplot
    ))

    ## Data

    # Loop over the three safety zones
    for maxval, minval, color, name in zip(
        [1,1,100], [0, 1, 2],
        ['#86A851', '#FAB800', '#F07204'],
        LEGEND_LABELS,
    ):
        # The values for the zone are added as polar bar plots, so incremental values,
        # which is why we need to substract the start value of the safety zone (minval)
        rvalues = (data-minval).clip(0, maxval)[column].values

        fig.add_trace(go.Barpolar(
            r=rvalues, theta=midthetas,
            width=360/n, marker_color=color,
            subplot=subplot2, marker_line_width=0, name=name, showlegend=subplot=='polar'
        ))


    _num = 100
    ## Circular grid lines
    # Safety zone, r=1
    if show_safety_grid:
        fig.add_trace(go.Scatterpolar(
            r=np.ones(_num)*1, theta=np.linspace(0,360,_num), showlegend=False,
            line={'color': 'black', 'dash': 'dot', 'width': 2}, subplot=subplot2
        ))
        # Uncertainty zone, r=2
        fig.add_trace(go.Scatterpolar(
            r=np.ones(_num)*2, theta=np.linspace(0,360,_num), showlegend=False,
            line={'color': '#B1AFB2', 'dash': 'dot', 'width': 2}, subplot=subplot2
        ))
    # Other lines
    grid_rs = [1.5] + list(np.arange(2.5, max_val-0.5, 0.5))
    for r in grid_rs:
        fig.add_trace(go.Scatterpolar(
            r=np.ones(_num)*r, theta=np.linspace(0,360,_num), showlegend=False,
            line_color='#E3E4E6', line_width=0.7, subplot=subplot2
        ))

    ## Category separators
    for i in range(n):
        theta = i * 360 / n
        dtheta = 0.7
        fig.add_trace(go.Scatterpolar(
            r=[max_val, 0., max_val], theta=[theta - dtheta, theta, theta + dtheta],
            fill='toself', mode='lines', fillcolor='black', line_width=0,
            showlegend=False, subplot=subplot2
        ))
        
    ## Return the layout values for this subplot
    return {
        subplot2: {
            'radialaxis_range': [0,max_val], 'hole': 0, 'radialaxis_visible': False,
            'angularaxis': {
                'rotation': 180, 'direction': 'clockwise',
                'showgrid': False, 'tickvals': midthetas, 'ticktext': category_names, 'ticks': 'inside', 'ticklen': 10
            },
            'bgcolor': 'rgba(0,0,0,0)',
            'domain': {'x': domain_x, 'y': domain_y}
        },
        subplot: {
            'domain': {'x': domain_x, 'y': domain_y},
            'radialaxis_range': [0,max_val], 'hole': 0, 'radialaxis_visible': False,
            'angularaxis_visible': False, 'bgcolor': 'rgba(0,0,0,0)'
        }
    }


In [3]:
dummy_data = pd.read_excel("dummy_data.xlsx", index_col=0)
dummy_data

,1970,2015,2030,2050,2100
Name,,,,,
Biosphere<br>integrity,2.008261,2.196283,2.274551,2.393406,2.649680
Climate<br>change,0.514019,1.825215,2.495222,3.168799,5.150088
Stratospheric<br>ozone,0.157970,0.902687,0.599751,0.326132,0.157970
Aerosols,1.800000,1.833333,1.766667,1.693333,NaN
Ocean<br>acidification,0.455323,0.813726,1.002747,1.649333,3.317681
Nitrogen<br>balance,1.337534,2.602915,2.916341,3.193354,3.386890
Phosphorous<br>balance,1.270859,2.067643,2.437663,2.447953,2.561381
Freshwater<br>use,0.573639,0.859729,0.941425,1.025775,1.025469
Land system<br>change,1.369600,1.539171,1.631971,1.727686,1.886345


In [4]:
fig = make_subplots(1, 1)

extra_layout = add_pb_subplot(dummy_data, 2050, fig)

fig.update_layout(
    width=800,
    **extra_layout
)

# Add lines to PB plot

In [5]:
COLOR_SEQUENCE = ['#00AEEF', '#B6036C', '#808D1D', '#FAAD1E', '#3F1464', '#7CCFF2',
       '#F198C1', '#42B649', '#EE2A23', '#004019', '#F47321', '#511607',
       '#BA8912', '#78CBBF', '#FFF229', '#0071BB']

def add_pb_subplot_lines(data, fig, reference=None, subplot='polar', subplot2='polar2', domain_x=[0,1], domain_y=[0,1], max_val=3.6, bg_limit = 3, ticks = 'inside', colors = COLOR_SEQUENCE, dashes=None, ref_color='#AAA', show_safety_grid=True, showbg=True):
    
    colormap = dict(zip(data.columns, colors))
    if dashes is None:
        dashmap = dict(zip(data.columns, ['solid']*len(data.columns)))
    else:
        dashmap = dict(zip(data.columns, dashes))
    n = len(data)
    beginthetas = np.arange(n) * 360 / n
    midthetas = beginthetas + 360/(2*n)
    category_names = data.index.values
    
    background_r = bg_limit

    ## Background
    if showbg:
        fig.add_trace(go.Barpolar(
            r=[background_r], theta=[0], width=[360], marker_color='#E9F0FA',
            showlegend=False, subplot=subplot
        ))
    
    _num = 100
    
    
    
    ## Circular grid lines
    # Safety zone, r=1
    if show_safety_grid:
        fig.add_trace(go.Scatterpolar(
            r=np.ones(_num)*1, theta=np.linspace(0,360,_num), showlegend=False,
            line={'color': 'black', 'dash': 'dot', 'width': 3}, subplot=subplot2
        ))
        # Uncertainty zone, r=2
        fig.add_trace(go.Scatterpolar(
            r=np.ones(_num)*2, theta=np.linspace(0,360,_num), showlegend=False,
            line={'color': '#B1AFB2', 'dash': 'dot', 'width': 3}, subplot=subplot2
        ))
    
    ## Reference
    if reference is not None:
        fig.add_barpolar(
            r=reference.values, theta=midthetas,
            width=360/n, marker_color=ref_color,
            subplot=subplot2, marker_line_width=0, name=reference.name, showlegend=subplot=='polar'
        )

    ## Data
    for i, midtheta in enumerate(midthetas):
        
        rounded = (data.iloc[i] * 9).round(decimals=0) / 9
        
        j = 0
        
        for r_round in rounded.unique():
            columns = rounded[rounded == r_round].index.values
            num_overlaps = len(columns)
            for k, (name, column) in enumerate(data[columns].items()):
                color = colormap[name]
                dash = dashmap[name]
                j += 1
                
                r = column.values[i]
                # if r > max_val:
                #     r = max_val - 0.01
                #     line_style = 'dot'
                # else:
                #     line_style = 'solid'
                line_style = dash

                begin_theta_full, end_theta_full = midtheta - 360/(2*n), midtheta + 360/(2*n)
                
                delta_theta = (end_theta_full - begin_theta_full) / num_overlaps
                begin_theta = begin_theta_full + k * delta_theta
                end_theta = begin_theta + delta_theta

                fig.add_scatterpolar(
                    r = np.ones(_num)*r, theta=np.linspace(begin_theta, end_theta, _num),
                    line={'color': color, 'width': 4, 'dash': line_style},
                    showlegend=False, name=name, subplot=subplot2
                )

    if subplot == 'polar':
        for name, color in colormap.items():
            dash = dashmap[name]
            fig.add_scatterpolar(
                r=[None], theta=[None], line={'color': color, 'width': 4, 'dash': dash}, name=name, subplot=subplot2, mode='lines',
            )
    
    
    
    # Other lines
    grid_rs = [1.5] + list(np.arange(2.5, max_val-0.5, 0.5))
    for r in grid_rs:
        fig.add_trace(go.Scatterpolar(
            r=np.ones(_num)*r, theta=np.linspace(0,360,_num), showlegend=False,
            line_color='#E3E4E6', line_width=0.7, subplot=subplot
        ))

    ## Category separators
    for i in range(n):
        theta = i * 360 / n
        dtheta = 0.7
        fig.add_trace(go.Scatterpolar(
            r=[max_val, 0., max_val], theta=[theta - dtheta, theta, theta + dtheta],
            fill='toself', mode='lines', fillcolor='black', line_width=0,
            showlegend=False, subplot=subplot2
        ))
        
    ## Return the layout values for this subplot
    return {
        subplot2: {
            'radialaxis_range': [0,max_val], 'hole': 0, 'radialaxis_visible': False,
            'angularaxis': {
                'rotation': 180, 'direction': 'clockwise',
                'showgrid': False, 'tickvals': midthetas, 'ticktext': category_names, 'ticks': ticks, 'ticklen': 10
            },
            'bgcolor': 'rgba(0,0,0,0)',
            'domain': {'x': domain_x, 'y': domain_y}
        },
        subplot: {
            'domain': {'x': domain_x, 'y': domain_y},
            'radialaxis_range': [0,max_val], 'hole': 0, 'radialaxis_visible': False,
            'angularaxis_visible': False, 'bgcolor': 'rgba(0,0,0,0)'
        }
    }



In [13]:
fig2 = make_subplots(1, 1)

# Add same subplot as before
extra_layout = add_pb_subplot(dummy_data, 2050, fig2, show_safety_grid=False)

# Add lines for other scenarios/years (here: 1970 and 2015)
add_pb_subplot_lines(dummy_data.loc[:,[1970, 2015]], fig2, colors=['mediumvioletred', 'blue'], dashes=['solid', '2px 2px'], show_safety_grid=False, showbg=False)

fig2.update_layout(
    width=800,
    **extra_layout
)
fig2